In [1]:
import warnings

warnings.filterwarnings("ignore")

from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.model_training.trainer import MLTrainer
from tsururu.model_training.validator import KFoldCrossValidator
from tsururu.models.boost import CatBoost
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

There are several main objects to look out for when working with the library:
1) `TSDataset`.
2) `Pipeline` and `Transformers`
3) `Strategy`.
4) `Model`.

### TSDataset

This class is needed to store data and meta-information about it.

To initialise it is necessary to submit the data in `pd.DataFrame` format and define some meta-information about roles that necessary for solving the task of time series forecasting: `id`, `date`, `target`.

In [3]:
df_path = "datasets/global/simulated_data_to_check.csv"

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


### Pipeline and Transformers

#### What kind of transformers are there?

Special attention should be paid to the `Transformer` class: the elements of the pipeline that are responsible for transforming the values of a series and generating features. `Pipeline` class is a wrapper over transformers which is needed to provide some additional methods and functions above them.

There are two types of transformers that are used to collect pipelines:
- `Union` transformers;
- `Sequential` transformers.

Below is a list of available Transformers: 
- `StandardScalerTransformer` *(Series4Series)*.
- `DifferenceNormalizer` *(Series4Series)*: subtract the previous value or divide by it.
- `LastKnownNormalizer` *(Features4Features)*: normalize all lags by the last known one: divide by it or subtract.

This three transformers provide flags `transform_features` / `transform_target`, that allow you to manipulate traits and targets separately and get different results from them.

Besides, __DifferenceNormalizer__ and __LastKnownNormalizer__ can be applied in two regimes: `delta` and `ratio`: in the first case, normalisation means subtracting the target value from the current value, and in the second, dividing by it.

- `LabelEncodingTransformer` and `OneHotEncodingTransformer` *(Series4Series)* - encoders for categorical features.
- `TimeToNumGenerator` and `DateSeasonsGenerator` *(Series4Series)* - generator for seasonal features by dates.
- `LagTransformer` *(Series4Features) - generator for lags. 

__!!!The lag transformer must necessarily be present in the sequential transformer, otherwise the features will not be generated.!!!__

Finally, to generate targets, you need to use `TargetGenerator`.

#### Transformers must be assembled in order!

The __SeriesToSeries__ transformers should come first, followed by the LagTransformer and TargetGenerator (__SeriesToFeatures__), and then the __FeaturesToFeatures__ transformers.

!!!Thus, `StandardScalerNormalizer` and `DifferenceNormalizer` should be before `LagTransformer` and `LastKnownNormalizer` after it!!!

#### How to build a Pipeline?

So, there are two ways to build a pipline from transformers: initialise the transformers of interest by hand or use a config in the form of a dictionary. Let's look at both ways.

In [5]:
standard_scaler = StandardScalerTransformer(
    transform_features=True,
    transform_target=True
)

lag = LagTransformer(lags=3)
date_lag = LagTransformer(lags=3)
id_lag = LagTransformer(lags=1)

target_generator = TargetGenerator()

date_seasons = DateSeasonsGenerator(
    seasonalities=["doy", "m", "wd"],
    from_target_date=True,
)

In [6]:
union_1 = UnionTransformer(transformers_list=[lag, target_generator])

seq_1 = SequentialTransformer(transformers_list=[standard_scaler, union_1], input_features=["value"])
seq_2 = SequentialTransformer(transformers_list=[date_seasons, date_lag], input_features=["date"])
seq_3 = SequentialTransformer(transformers_list=[id_lag], input_features=["id"])

union = UnionTransformer(transformers_list=[seq_1, seq_2, seq_3])

In [7]:
pipeline_1 = Pipeline(union, multivariate=False)

In [8]:
pipeline_1.__dict__

{'transformers': <tsururu.transformers.base.UnionTransformer at 0x309876e30>,
 'multivariate': False,
 'is_fitted': False,
 'strategy_name': None,
 'output_features': None,
 'y_original_shape': None}

Or:

In [9]:
pipeline_params = {
    "target": {
        "columns": ["value"],
        "features": {
            "StandardScalerTransformer":
                {
                    "transform_target": True, 
                    "transform_features": True
                },
            "LagTransformer": {"lags": 7},
        },
    },
    "date": {
        "columns": ["date"],
        "features": {
            "DateSeasonsGenerator": {
                # Use seasonality features from the date column as 
                # features with datetime lags
                # Possible values: [
                #    "y": year, "m": month, "d": day, 
                #    "wd": weekday, "doy": dayofyear,
                #    "hour": hour, "min": minute, "sec": second, 
                #    "ms": microsecond,  "ns": nanosecond
                # ]
                "seasonalities": ['doy', 'm', 'wd'], 
                # Use date from target point to make datetime features
                "from_target_date": True,
            },
            "LagTransformer": {"lags": 3}
        },
    },
    "id": {
        "columns": ["id"],
        "features": {
            "LagTransformer": {"lags": 1},
        },
    }
}

In [10]:
pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

Or:

In [11]:
pipeline_easy_params = {
    # One from ["none", "standard_scaler", "difference_normalizer", "last_known_normalizer"]
    "target_normalizer": "standard_scaler",

    # One from ["none", "delta", "ratio"]  (MUST BE "none" for "standard_scaler" and NOT "none" for others)
    "normalizer_regime": "none",

    # One from ["features", "target", "features_target"]
    "normalizer_transform_regime": "features_target",

    "target_lags": 3,
    "date_lags": 1,
#    "exog_lags": 1,
}

In [12]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

#### Can I use exogenous variables in the pipeline?

Yes! Exogenous variables can also be specified here. Just include them in your pipeline.

However, their operation is currently tested only for the `MIMOStrategy` in global-modelling. For other strategies support of additional variables is under development.

In [13]:
# pipeline_params["exog_group_1"] = {
#     "columns": ["value"],
#     "features": {
#         "StandardScalerTransformer":
#             {
#                 "transform_target": False, 
#                 "transform_features": True
#             },
#         "LagTransformer": {"lags": 7},
#     },
# }

__Make sure you have the transform_target = False flag for exogenous features!__

#### Model, Validator and Trainer

- `Model`:
  - The model is separate from the strategy. Any model can be run in any strategy if it supports this input and output format.
  - You can use on of the implemented ML models (for instance, GBM (Gradient Boosting Machine)).
- `Validator`:
  - The validator is responsible for setting up the validation process, which includes creating training and validation folds. It ensures that the data is split correctly so that the model’s performance can be accurately assessed. 
- `Trainer`:
  - The trainer is the component that trains the model with provided validator. 
  - It is necessary to choose a trainer in accordance with the type of model (ML, DL, stats).

In [14]:
# Configure the model parameters
model = CatBoost
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

trainer_params = {}

trainer = MLTrainer(
    model,
    model_params,
    validation,
    validation_params,
)

#### Strategy

- _Recursive:_ 
    - one model for all points of the forecast horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: a prediction is iteratively made one point ahead, and then this prediction is used to further shape the features in the test data. 
- _Recursive-reduced:_
    - one model for all points in the prediction horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: features are generated for all test observations at once, unavailable values are replaced by NaN.
- _Direct:_ 
    - individual models for each point in the prediction horizon. 
- _MultiOutput (MIMO - Multi-input-multi-output):_
    - one model that learns to predict the entire prediction horizon. 
    - __Also, this strategy supports the presence of `exogenous features` (only for local- or global-modelling).__
- _FlatWideMIMO:_.
    - mixture of Direct and MIMO, fit one model, but uses deployed over horizon Direct's features.
    - __Number of `lags for datetime features` should be equal to `horizon` while using this strategy.__

In [15]:
horizon = 3
history = 7
step = 1

In [16]:
strategy = RecursiveStrategy(horizon, history, step, trainer, pipeline)

In [17]:
fit_time, _ = strategy.fit(dataset)

0:	learn: 0.9603852	test: 0.9667066	best: 0.9667066 (0)	total: 56.5ms	remaining: 56.4s
500:	learn: 0.0043385	test: 0.0045465	best: 0.0045465 (500)	total: 420ms	remaining: 418ms
999:	learn: 0.0027344	test: 0.0030471	best: 0.0030471 (999)	total: 763ms	remaining: 0us

bestTest = 0.003047062205
bestIteration = 999

Fold 0. Score: 0.003047062205400275
0:	learn: 0.9660362	test: 0.9613700	best: 0.9613700 (0)	total: 1.65ms	remaining: 1.65s
500:	learn: 0.0043259	test: 0.0046427	best: 0.0046427 (500)	total: 342ms	remaining: 340ms
999:	learn: 0.0026124	test: 0.0029169	best: 0.0029169 (999)	total: 705ms	remaining: 0us

bestTest = 0.002916854444
bestIteration = 999

Fold 1. Score: 0.002916854443892958
Mean score: 0.003
Std: 0.0001


In [18]:
forecast_time, current_pred = strategy.predict(dataset)

In [19]:
current_pred

,id,date,value
0,0,2022-09-27,1997.088714
1,0,2022-09-28,1997.191979
2,0,2022-09-29,1995.496356
3,1,2022-09-27,2997.475196
4,1,2022-09-28,2997.553282
5,1,2022-09-29,2995.826568
6,2,2022-09-27,3997.392796
7,2,2022-09-28,3997.46765
8,2,2022-09-29,3995.743526
9,3,2022-09-27,4997.548721


## Backtest validation of pipeline

In [20]:
ids, test, pred, fit_time, forecast_time = strategy.back_test(dataset, cv=1)

0:	learn: 0.9616026	test: 0.9653712	best: 0.9653712 (0)	total: 1.06ms	remaining: 1.06s
500:	learn: 0.0042762	test: 0.0045139	best: 0.0045139 (500)	total: 371ms	remaining: 370ms
999:	learn: 0.0026729	test: 0.0029726	best: 0.0029726 (999)	total: 747ms	remaining: 0us

bestTest = 0.002972644218
bestIteration = 999

Fold 0. Score: 0.0029726442175072565
0:	learn: 0.9646266	test: 0.9624752	best: 0.9624752 (0)	total: 1.12ms	remaining: 1.12s
500:	learn: 0.0043855	test: 0.0045979	best: 0.0045979 (500)	total: 386ms	remaining: 384ms
999:	learn: 0.0026660	test: 0.0029718	best: 0.0029718 (999)	total: 748ms	remaining: 0us

bestTest = 0.002971764957
bestIteration = 999

Fold 1. Score: 0.002971764957214351
Mean score: 0.003
Std: 0.0


In [21]:
get_results(cv=1, regime="global", y_true=test, y_pred=pred, ids=ids)

,y_true_1,y_pred_1,id_1
0,1997.0,1993.68704,0
1,1998.0,1994.202883,0
2,1999.0,1994.669445,0
3,2997.0,2993.991901,1
4,2998.0,2994.586288,1
5,2999.0,2995.082926,1
6,3997.0,3993.914717,2
7,3998.0,3994.509697,2
8,3999.0,3995.015013,2
9,4997.0,4994.022114,3


## Working with raw time series' granularity

Time series come in different granularities, from hourly and daily time series to more complex ones such as the end of each quarter.

If the rows do not contain segments that are too short (that are shorter than history + horizon), then `tsururu` will try to extract the row granularity on its own. We currently support the following types:

- Yearly (and YearlyEnd)
- Quarterly (and Quarterly)
- Monthly (and MonthlyEnd)
- Weekly
- Daily
- Hourly
- Minlutely
- Secondly
- Microsecondly

There is also support for compound granularities (10 days, 15 minutes, 32 seconds, etc.). The correctness of the selected granularity can be checked from the output after the `Dataset` class has been created.

However, there are tricky situations (e.g. 28 days) where the monthly granularity may be guessed incorrectly. Therefore, it is possible to set your own granularity using the `pd.DateOffset` class or related classes from `pandas.tseries.offsets`, which must be fed as `delta` parameter into the `Dataset` class. Then the time column will be processed according to the user's settings.

Consider a time series where each point is exactly __28 daily points away__ from each other

In [22]:
df_path_2 = "datasets/global/simulated_data_to_check_28D.csv"

# Configure the features settings
dataset_params_2 = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [23]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    print_freq_period_info=True,
)

freq: Month; period: 1.0


We see that the frequency of the series is incorrectly defined as monthly. Let's try to pass the `delta` parameter.

In [24]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    delta=pd.DateOffset(days=28),
    print_freq_period_info=True,
)

Custom OffSet: <DateOffset: days=28>


Now it's all detected correctly.